In [1]:
# iPython widgets, alle possibilities:
from IPython.html import widgets
from ipywidgets import *
from IPython.display import display
import pandas as pd
import unirest
import fuzzyTimeRatio as fuzzyTime
import cluster_recipes as cluster
import defuzzifyInputs as di
import heapq
from IPython.display import clear_output

def doTheSearch(button):
    clear_output()
    # call from api
    url = "https://spoonacular-recipe-food-nutrition-v1.p.mashape.com/recipes/search?number=15&offset=0&query=" 
    url += searchterms.value +"&type=main+course&cuisine=" + cuisine.value
    recipes = unirest.get(url,
      headers={
        "X-Mashape-Key": "uI9T1GTt8nmshcUUWJOjq8TQNGBgp1P9Zffjsn7dAbkmTSDt1k",
        "Accept": "application/json"
      }
    )

    # present data properly
    testData = recipes.body
    testFrame = pd.DataFrame(testData['results'], columns=['id', 'title', 'readyInMinutes'])
    print testFrame
    # fuzzify ratio
    degreeRatios = []
    ratios = testFrame.get('readyInMinutes')/int(time_available.value)
    for ratio in ratios: 
        _, degreesRatio = fuzzyTime.member_of_function(ratio)
        degreeRatios.append(degreesRatio)

    degreeClusters = []
    # fuzzify activity
    recipeIds = list(testFrame.id.values)
    for i in range(len(recipeIds)):
        idRecipe = recipeIds[i]
        clusteringResponse = cluster.testClustering(4, 1.5, str(idRecipe))
        # if recipe id exists
        if clusteringResponse != 0:
            _, clusters = clusteringResponse
            degreeClusters.append(clusters)
        else: 
            degreeRatios.pop(i)
    
    crisp_outputs = []
    for i in range(len(degreeRatios)): 
        degreesRatio = degreeRatios[i]
        degreesClusters = degreeClusters[i]
        output = di.defuzzify(degreesRatio, degreesClusters, activities, activity.value)
        crisp_outputs.append(output)
    
    sortedIndices = heapq.nlargest(len(crisp_outputs), xrange(len(crisp_outputs)), key=crisp_outputs.__getitem__)[:5]
    rankedRecipes = testFrame.ix[sortedIndices]
    
    print rankedRecipes['title']

activities = ['Post-workout', 'Study', 'Life', 'Workout']
searchterms = widgets.Text(description='Keywords: ', margin = '10px')
cuisine = widgets.Text(description='Cuisine: ', margin = '10px')
time_available = widgets.IntText(description='Time Available: ', margin = '10px')
activity = widgets.Dropdown(options=activities, description='What are you going to do?:', margin = '10px', button_style='info')

tab1_children = [searchterms, cuisine, time_available, activity]

tab1 = VBox(children=tab1_children)

i = widgets.Accordion(children=[tab1], border_color='black', margin = '10px')

i.set_title(0,"Search for a recipe!")
button = widgets.Button(description="Search", icon='check', button_style='success', width='100px', margin = '10px')
display(i)
display(button)
button.on_click(doTheSearch)

       id                                              title  readyInMinutes
0  834986                          Skinny Chicken Enchiladas              45
1  831353  Slow Cooker Chicken Burrito Bowls with Spirali...              45
2  832819                     Marinated Chicken Fajita Salad              30
3  835345                      Honey Lime Chicken Enchiladas              50
4  833362                          Mexican Chicken Casserole              30
5  834059                        Creamy Chicken Fajita Pasta              25
5                          Creamy Chicken Fajita Pasta
4                            Mexican Chicken Casserole
2                       Marinated Chicken Fajita Salad
0                            Skinny Chicken Enchiladas
1    Slow Cooker Chicken Burrito Bowls with Spirali...
Name: title, dtype: object
